In [1]:
import os
import pandas as pd
from pathlib import Path
from moviepy.editor import VideoFileClip
import re

In [3]:
def extract_segments(csv_path, video_path, output_dir, camera_name, timestamp):
    df = pd.read_csv(csv_path)
    
    people_counts = df['People Count'].tolist()
    times = df['Time (s)'].tolist()
    frames = df['Frame'].tolist()
    
    segments = []
    start_idx = None
    people_streak = 0
    zero_streak = 0
    max_people = 0
    
    for i in range(len(people_counts)):
        if people_counts[i] > 0:
            people_streak += 1
            zero_streak = 0
            max_people = max(max_people, people_counts[i])
        else:
            zero_streak += 1
            if zero_streak > 5:
                if start_idx is not None:
                    segments.append((frames[start_idx], times[start_idx], times[i], max_people))
                    start_idx = None
                    max_people = 0
                zero_streak = 0
                people_streak = 0
        
        if people_streak > 5 and start_idx is None:
            start_idx = i - 5
    
    if start_idx is not None:
        segments.append((frames[start_idx], times[start_idx], times[-1], max_people))
    
    if not segments:
        return
    
    os.makedirs(output_dir, exist_ok=True)
    
    with VideoFileClip(video_path) as video:
        print("segments for ", timestamp)
        for idx, (frame, start, end, max_people) in enumerate(segments, 1):
            output_file = os.path.join(output_dir, f"{camera_name}_t{timestamp}_f{frame}_p{max_people}_{idx}.mp4")
            
            clip = video.subclip(start, end)
            clip.write_videofile(output_file, codec='libx264', audio_codec='aac', verbose=False, logger=None)
            print(f"Saved segment: {output_file}")

In [4]:
def main():
    recordings_dir = "recordings"
    timestamps_dir = "timestamps"
    cropped_dir = "mp4crop"
    
    for cam in range(1, 6):
        cam_folder = f"camera{cam}"
        cam_path = os.path.join(recordings_dir, cam_folder)
        out_path = os.path.join(cropped_dir, cam_folder)
        timestamp_path = os.path.join(timestamps_dir, cam_folder)
        
        if not os.path.exists(cam_path) or not os.path.exists(timestamp_path):
            continue
        
        for file in os.listdir(cam_path):
            if file.endswith(".mp4"):
                video_path = os.path.join(cam_path, file)
                csv_path = os.path.join(timestamp_path, Path(file).stem + ".csv")
                
                match = re.search(r"camera\d+_(\d{8})_(\d{6})", file)
                if match:
                    timestamp = match.group(2)  # Получаем HHMMSS из названия файла
                else:
                    timestamp = "000000"
                
                if os.path.exists(csv_path):
                    extract_segments(csv_path, video_path, out_path, cam_folder, timestamp)

In [ ]:
if __name__ == "__main__":
    main()

segments for  043858
Saved segment: mp4crop/camera1/camera1_t043858_f155_p1_1.mp4
Saved segment: mp4crop/camera1/camera1_t043858_f410_p1_2.mp4
Saved segment: mp4crop/camera1/camera1_t043858_f565_p1_3.mp4
Saved segment: mp4crop/camera1/camera1_t043858_f680_p5_4.mp4
segments for  094552
Saved segment: mp4crop/camera1/camera1_t094552_f25_p4_1.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f390_p2_2.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f1100_p3_3.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f1380_p1_4.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f1940_p5_5.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f7325_p1_6.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f7610_p3_7.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f7885_p3_8.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f8560_p3_9.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f8900_p2_10.mp4
Saved segment: mp4crop/camera1/camera1_t094552_f9610_p2_11.mp4
Saved segment: mp4crop/camera1/ca